allora raga vi faccio una piccola guida sul codice di merda che ho scritto finora così non dovete decifrarlo per iniziare a lavorarci



io ho sempre fatto tutto su colab, quindi ecco il link per condividere il dataset che ho caricato su drive, così nel notebook su colab uso "gdown" che scarica da drive e ci mette un secondo e non devo più perdere tempo a caricare manualmente il dataset volta per volta (ero scemo). 

link:

https://drive.google.com/drive/folders/1-6PivWKBG4uVCSJ9pbLZMdI-nIhjaOcf?usp=drive_link

https://drive.google.com/drive/folders/1eDZL1LzKCbRM9clKK1DWItzT0MSVoiG1?usp=drive_link

https://drive.google.com/drive/folders/1QxeM1yzZFUYX9uDZZsCACxW77sZijlFE?usp=drive_link

bho raga non so come funziona drive in teoria basta il primo link, gli altri due sono i link alle cartelle "images" e "annotations", il primo è il link alla cartella root "dataset" dovrebbe funzionare, lo cliccate così ci potete accedere dai vostri account google.

tenete ON_COLAB a True e USE_DRIVE a False

la funzione per visualizzare il data augmention l'ho aggiustata, prima la terza riga era buggata. la versione nuova è in *train_resnet_modifica.ipynb*

### la funzione di training 

è abbastanza semplice basta leggere il commento nella funzione, la funzione **lr_scheduler_fn** però attenti che deve prendere in input current learning rate, e lista delle val accuracies, perchè nella funzione di train lr_scheduler_fn la chiamo con quei due parametri. poi può avere altri parametri opzionali ma non vengono settati di base nella funzione di train

il modello usato di base è una resnet18 alla quale però ho abbassato parecchio il numero di canali, all'inizio partivano da 64 e arrivavano a 512 e aveva 11 milioni di parametri, ora l'ho fatta partire da 16 (700k parametri) e ha la stessa performance. ho abbassato anche a partire da 8 e non è che peggiora molto. considerando che il dataset sono solo 3700 immagini ci sta che non serva un modello troppo grosso

ho provato ad aggiungere squeeze excitate (SE) ma secondo me sono troppo sofisticati e va in overfit di brutto, infatti si vede dal log dei training

ho anche aggiunto una cosa chiamata Feature Pyramidal Network (FPN) che è una cosa che non conoscevo consigliata da copilot. praticamente connette alla cazzo layer su layer e non serve a un cazzo perchè tanto non è migliorata

io vi direi di partire da modello da 700k che vi lascio in train_resnet_modifica.ipynb e magari provare a dimezzare i canali e aggiungere altre cose oppure sbizzarritevi come volete

per il data augmentation cercate nel codice transform_data_augmentation

nello scheduler ho aggiunto delle pazienze un po' particolari:

questa riga che è un po' confusionaria

if max(recent_accuracies) + (previous_max / 200) <= previous_max:

praticamente significa: se il massimo delle val acc delle ultime 5 epoche, PIÙ un piccolo margine (considerate che sta intorno all'1% se non meno) è minore del massimo di tutte le epoche tranne le ultime 5, allora la validation accuracy è stallata ed è ora di aggiornare il learning rate

dopo il training il file contiene una cella per salvare log di metriche e pesi del modello e una cella che misura la accuracy sul test set (deve arrivare a 60%). e anche una cella che dovrebbe fare il disegnino della rete ma non è fondamentale